In [1]:
import sys
sys.path.append("../../../h2o-py/build/main")

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
import h2o
from h2o.estimators import H2ODecisionTreeEstimator
h2o.init()

versionFromGradle='3.46.0',projectVersion='3.46.0.99999',branch='valenad-test-test',lastCommitHash='d7daf66d53941dbd75639a0ec6ffe1b2910de62a',gitDescribe='jenkins-3.46.0.2-1-gd7daf66d53',compiledOn='2024-05-24 14:11:07',compiledBy='potte'
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 mins 21 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.99999
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_potte_gzdbgd
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,23.91 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


# Algoritm to split preprocest datasets

In [4]:
titanic = h2o.import_file('train-prep.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
titanic

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,nan,7.25,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,nan,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,nan,7.925,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,S
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,C
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,nan,16.7,S


In [6]:
train, valid = titanic.split_frame(ratios=[.7])

In [7]:
titanic = titanic.as_data_frame()
train = train.as_data_frame()
valid = valid.as_data_frame()

D:\skola\dip\h2o-3\_valenad-notebooks\machine-learning-experiments\H2O\../../../h2o-py/build/main\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [8]:
valid

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,NaN,71.2833,C
1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450.0,8.0500,S
2,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909.0,21.0750,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,NaN,16.7000,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783.0,26.5500,S
...,...,...,...,...,...,...,...,...,...,...,...
192,857,1,1,"Wick, Mrs. George Dennick (Mary Hitchcock)",female,45.0,1,1,36928.0,164.8667,S
193,861,0,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026.0,14.1083,S
194,877,0,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534.0,9.8458,S
195,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552.0,10.5167,S


In [9]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,NaN,7.2500,S
1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,NaN,7.9250,S
2,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803.0,53.1000,S
3,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463.0,51.8625,S
4,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742.0,11.1333,S
...,...,...,...,...,...,...,...,...,...,...,...
510,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,NaN,7.0500,S
511,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652.0,29.1250,Q
512,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536.0,13.0000,S
513,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053.0,30.0000,S


In [10]:
valid.to_csv('valid-split.csv',index = False)
train.to_csv('train-split.csv',index = False)